In [1]:
import numpy as np
import pandas as pd

In [ ]:
book = pd.read_csv('book.csv')
RUB = pd.read_csv('rub.csv')
user = pd.read_csv('user.csv')

# 데이터 전처리

### 1. 도서 데이터

In [ ]:
# 중복 도서 처리
book[book.duplicated(subset='Title')][book[book.duplicated(subset='Title')]['Title']=='데미안']
book_id_set = book[['Book_id','Title']]
RUB = pd.merge(RUB,book_id_set,on='Book_id')
book_nonduplicated = book.drop_duplicates(subset='Title')

In [ ]:
# 도서 분야 '분야 3'으로 정리
book_Genre_set = book[['Book_id','분야_3']]
RUB_Genre = pd.merge(RUB,book_Genre_set,on='Book_id')
genre_list = list(book['분야_3'].dropna().unique())
genre_list = sorted(genre_list, key=str)
df_genre = pd.DataFrame(columns = genre_list)
df_genre    #결과확인

### 2. 유틸리티 행렬 생성

In [ ]:
# 도서 유틸리티 행렬 생성 함수
def book_list():
    book_list = list(book['Title'].unique())
    book_list = sorted(book_list, key=str)
    df_matrix = pd.DataFrame(columns = ['id'] + book_list)
    user_num = RUB['id'].unique()
    
    book_num = len(RUB['Title'].unique())
    book_list = np.array(book_list)

    user_watch_list = []    # df_matrix에서 'user' 컬럼을 제외한 컬럼. 즉, title 컬럼
    for num in user_num:
        user_watch_list = [0 for i in range(book_num)]    # 값을 0으로 초기화
        df_tmp = RUB[RUB['id'] == num]    # 각 유저별로 묶는다
    
        for j in df_tmp.index:
            i = np.where(book_list==df_tmp.Title[j])[0][0]
            user_watch_list[i] = 1 # user가 보거나/보는중/보고싶다고 했다면 1 채워넣는다
        
            df_matrix.loc[num] = [num] + user_watch_list    # 행채우기]
    df_matrix = df_matrix.rename(columns={13.67:'13.67'})
    df_matrix = df_matrix.drop('id',axis=1)
    return df_matrix

In [ ]:
# 유저 profile 테이블 생성 함수
def profile_list():
    user_profile = user[['id','직업/세부','나이']]
    user_profile = user_profile.set_index('id')
    return user_profile

In [ ]:
# 장르 유틸리티 행렬 생성 함수
def genre_list():
    genre_list = list(book['분야_3'].dropna().unique())
    genre_list = sorted(genre_list, key=str)
    df_genre = pd.DataFrame(columns = genre_list)
    
    genre_list = list(book['분야_3'].dropna().unique())
    genre_list = sorted(genre_list, key=str)
    df_genre_rate = pd.DataFrame(columns = genre_list)
    
    user_num = RUB['id'].unique()
    genre_num = len(RUB['분야_3'].dropna().unique())
    genre_list = np.array(genre_list)

    user_genre_list = []    # df_matrix에서 'user' 컬럼을 제외한 컬럼. 즉, title 컬럼
    for num in user_num:
        user_genre_list = [0 for i in range(genre_num)]    # 값을 0으로 초기화
        df_tmp = RUB[RUB['id'] == num]
        #print(df_tmp.head(5))
        for j in df_tmp.index:
            df_tmp = df_tmp.fillna(value='null')
            try:
                i = np.where(genre_list==df_tmp.분야_3[j])[0][0]
                user_genre_list[i] = user_genre_list[i]+1 # user가 보거나/보는중/보고싶다고 했다면 1 채워넣는다
            except:
                continue
            df_genre.loc[num] = user_genre_list    # 행채우기
    for i in df_genre.index:
        tmp=[]
        tmp = df_genre.loc[i]/df_genre.sum(axis=1).loc[i]
        df_genre_rate.loc[i] = tmp
    return df_genre_rate

# Gower Similarity Measure

In [ ]:
# 독서 기록에 따른 거리 측정 함수
def history_dist(name):
    sum_list=[]
    history_dist = pd.DataFrame(df_matrix.index,columns=['id'])
    for i in tqdm(df_matrix.index):
        sum_dist=0
        for j in df_matrix.columns:
            if (df_matrix.loc[510][j]==1)&(df_matrix.loc[i][j]==1):
                dist=0
            else:
                dist=1
            sum_dist += dist
        sum_list.append(sum_dist)
    history_dist['ht_distance'] = sum_list
    return history_dist

In [ ]:
# 유저 프로필에 따른 거리 측정 함수
def profile_dist(name):
    user_profile.fillna(value=0,inplace=True)
    MM = max(user_profile['나이']) - min(user_profile['나이'])
    sum_list_2 = []
    profile_dist = pd.DataFrame(user_profile.index)
    
    for i in user_profile.index:
        sum_dist = 0
        if user_profile.loc[name]['직업/세부'] == user_profile.loc[i]['직업/세부']:
            dist=0
        else:
            dist=1
        
        dist_2 = abs(user_profile.loc[name]['나이'] - user_profile.loc[i]['나이'])/MM
        
        sum_dist = sum_dist + dist + dist_2
        sum_list_2.append(sum_dist)
    profile_dist['pf_distance'] = sum_list_2
    return profile_dist

In [ ]:
# 나이 min-max normalize
%%time

MM = max(user_profile['나이']) - min(user_profile['나이'])

sum_list_2 = []
profile_dist = pd.DataFrame(user_profile.index)

for i in user_profile.index:
    sum_dist = 0
    if user_profile.loc[510]['직업/세부'] == user_profile.loc[i]['직업/세부']:
        dist=0
    else:
        dist=1

    dist_2 = abs(user_profile.loc[510]['나이'] - user_profile.loc[i]['나이'])/MM
    
    sum_dist = sum_dist + dist + dist_2
    sum_list_2.append(sum_dist)
profile_dist['pf_distance'] = sum_list_2

In [ ]:
# 장르 count 비율로 산출
genre_list = list(book['분야_3'].dropna().unique())
genre_list = sorted(genre_list, key=str)
df_genre_rate = pd.DataFrame(columns = genre_list)
df_genre_rate  #결과확인

In [ ]:
# 종합 거리 산출: 'name' 유저와 근접한 'a'명이 가장 많이 등록한 책 'b'권을 추천
def RC_function(name, a, b): #name의 사람, a명의 사람
    rank = hd.merge(pt).merge(gd)
    rank['distance'] = (rank['gr_distance']+ rank['pf_distance']+ rank['ht_distance'])/(19522+2+1)
    rank = rank.sort_values(by='distance')[1:a]
    
    read = df_matrix.loc[[name]].T
    rank_user=[]
    for i in df_matrix.index:
        rank_user.append(i in list(rank['id']))
    df_matrix[rank_user].T.sum(axis=1)
    
    read['rank'] = df_matrix[rank_user].T.sum(axis=1)
    read.sort_values(by='rank',ascending=False)
    book_rank = read.sort_values(by='rank',ascending=False)
    result = book_rank[book_rank[name]==0][:b]
    
    return result.index

In [ ]:
# 매트릭스 생성
df_matrix = book_list()
user_profile = profile_list()
df_genre_rate = genre_list()

In [ ]:
# 거리 계산
hd = history_dist(510)
pt = Profile_dist(510)
gd = genre_dist(510)

In [ ]:
# 도서 추천
list(RC_function(510, 50, 5))
# ['사피엔스 - 유인원에서 사이보그까지, 인간 역사의 대담하고 위대한 질문','데미안', '총 균 쇠 (반양장) - 무기.병균.금속은 인류의 운명을 어떻게 바꿨는가, 
#   개정증보판','아주 작은 습관의 힘 - 최고의 변화는 어떻게 만들어지는가','나미야 잡화점의 기적']